In [1]:
import matlab.engine
import time

In [2]:
eng = matlab.engine.start_matlab()

In [10]:
eng.quit()

In [3]:
print(eng.isprime(37))

True


In [4]:
state = eng.default_state_PSS()

In [21]:
start_time = time.time()
for _ in range(20):
    state = eng.sim_step(state, 1.)
print("--- %s seconds ---" % (time.time() - start_time))

--- 163.74731183052063 seconds ---


In [5]:
start_time = time.time()
for _ in range(100):
    state = eng.sim_step(state, 1.)
print("--- %s seconds ---" % (time.time() - start_time))

MatlabExecutionError: Initial state vector "X0" must be a real vector of length 1
